In [18]:
import os

# Set the proxy environment variables
os.environ['http_proxy'] = 'http://127.0.0.1:7890'
os.environ['https_proxy'] = 'http://127.0.0.1:7890'

In [2]:
import ee
import geemap
import pprint

In [3]:
ee.Initialize()

### Define basic parameters

In [5]:
Region_cn = '华北'
Region_en = 'huabei'

In [6]:
# import the region_shp
Region_shp = ee.FeatureCollection("users/wangjinzhulala/China_built_up/01_Boundary_shp/China_zone")\
                     .filterMetadata('NAME1','equals',Region_cn)

In [7]:
# import the 20km_grid; We will randomly select one built_pt form it for validation purpose
Region_20km_grid = ee.FeatureCollection("users/wangjinzhulala/China_built_up/01_Boundary_shp/China_zone_grid_20km")\
                     .filterMetadata('NAME1','equals',Region_cn)

In [8]:
# import the China_ROI_Rectange; We will randomly select one built_pt form it for img_classification purpose
ROI_Rectange = ee.FeatureCollection("users/wangjinzhulala/China_built_up/01_Boundary_shp/China_ROI_rectangle")\
                 .filterBounds(Region_20km_grid)

In [28]:
# import the Administration points of China; 
Admin_pt = ee.FeatureCollection("users/wangjinzhulala/China_built_up/01_Boundary_shp/China_villages_30k")\
             .filterBounds(Region_20km_grid)

### For each rectangle, only select one point for all Admin_pt within 

##### Define spatial join 

In [30]:
# Define a spatial filter, with distance 10 m.
Intersect_Filter = ee.Filter.intersects(leftField  = '.geo',
                                        rightField = '.geo',
                                        maxError   = 10)

In [31]:
# Define a join, save all points.
Intersect_SaveAll = ee.Join.saveAll(matchesKey = 'points')

##### First get the Built_up_pts constrained by China_Rec_20km

In [32]:
# define a fuction that get the points out of the rectangle and only chose one of them
def chose_one_pt(fe):
    
    pts = ee.List(fe.get('points'))
    one_pt = ee.FeatureCollection(pts).randomColumn().sort('random').first()  
    
    return one_pt

In [33]:
# Apply the intersect_join
China_Rec_intersect_point = Intersect_SaveAll.apply(Region_20km_grid,Admin_pt,Intersect_Filter)

# only select one point from the intersection
China_Rec_intersect_point_one = China_Rec_intersect_point.map(lambda fe: chose_one_pt(fe))

In [34]:
Num_China_20km_constrained = China_Rec_intersect_point_one.size().getInfo()

##### Using the Num_China_20km_constrained to limit the number of ROI_Rectangle

In [36]:
ROI_Rectange_fit_China_20km = ROI_Rectange.randomColumn().sort('random').limit(Num_China_20km_constrained)

In [37]:
# get the ROI_constraiend points
ROI_intersect_point     = Intersect_SaveAll.apply(ROI_Rectange_fit_China_20km,Admin_pt,Intersect_Filter)
ROI_intersect_point_one = ROI_intersect_point.map(lambda fe: chose_one_pt(fe))

##### Remove duplicate points

In [38]:
# define a invert join
invertedJoin = ee.Join.inverted()

# remove the duplicated points in China_Rec_intersect_point_one (compare to ROI_intersect_point_one)
China_Rec_not_intersect_ROI = invertedJoin.apply(China_Rec_intersect_point_one,ROI_intersect_point_one, Intersect_Filter)

# merge ROI_not_intersect_China_Rec to China_Rec_intersect_point_one
Full_Built_points = China_Rec_not_intersect_ROI.merge(ROI_intersect_point_one)

In [61]:
for pt,name in zip([China_Rec_not_intersect_ROI,ROI_intersect_point_one],
                    ['Built_pt_China_20km','Built_pt_ROI']):  


    # define export variables
    name        = f'02_{name}_{Region_en}'
    export_path = 'users/wangjinzhulala/China_built_up/02_control_sample'

    # exporting to gee aeest
    task = ee.batch.Export.table.toAsset(   collection  = pt,
                                            description = f'{name}',
                                            assetId     = f'{export_path}/{name}')
    task.start()

### Download the points to local drive

In [39]:
Built_pt_China_20km = ee.FeatureCollection(f"users/wangjinzhulala/China_built_up/02_control_sample/02_Built_pt_China_20km_{Region_en}")
Built_pt_ROI = ee.FeatureCollection(f"users/wangjinzhulala/China_built_up/02_control_sample/02_Built_pt_ROI_{Region_en}")

In [15]:
for pt,name in zip([Built_pt_China_20km,Built_pt_ROI],
                   ['Built_pt_grid_20km','Built_pt_ROI']):
    
    geemap.ee_to_shp(pt, filename= f'./Raw_shp/{name}_{Region_en}.shp')

Generating URL ...
Please wait ...
[WinError 32] The process cannot access the file because it is being used by another process: 'C:\\Users\\wangj\\OneDrive\\图片\\National_Built_up_map\\Process_2_Classification_process\\Sub_Process_6_Sample_creation_Built\\Raw_shp\\Built_pt_grid_20km_huabei.zip'
Generating URL ...
Please wait ...
[WinError 32] The process cannot access the file because it is being used by another process: 'C:\\Users\\wangj\\OneDrive\\图片\\National_Built_up_map\\Process_2_Classification_process\\Sub_Process_6_Sample_creation_Built\\Raw_shp\\Built_pt_ROI_huabei.zip'
